<a href="https://colab.research.google.com/github/kani215/ML_study/blob/main/ml8_%EC%95%99%EC%83%81%EB%B8%94_%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 8.1 투표 기반 분류기 
- 로지스틱 회귀 분류기
- svm 분류기
- 랜덤 포레스트 분류기
- kmeans 분류기
중 가장 잘 맞춘 녀석을 선택하는 앙상블 수법

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

log_clf=LogisticRegression()
rnd_clf=RandomForestClassifier()
svm_clf=SVC()

X,y=make_moons(n_samples=500, noise=0.30, random_state=42)
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42)

voting_clf=VotingClassifier(
    estimators=[('lr',log_clf),('rf',rnd_clf),('svc',svm_clf)],
    voting='hard')
voting_clf.fit(X_train,y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [5]:
from sklearn.metrics import accuracy_score
for clf in (log_clf,rnd_clf,svm_clf,voting_clf):
  clf.fit(X_train,y_train)
  y_pred=clf.predict(X_test)
  print(clf.__class__.__name__,accuracy_score(y_test,y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.896
SVC 0.896
VotingClassifier 0.904


8.2 배깅과 페이스팅 기법
- 예제는 500개의 분류기를 앙상블 훈련 시키는 코드
- 각 분류기는 무작위로 100개 샘플로 추려 그거를 훈련한 것 

In [6]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators = 500,
    max_samples=100, bootstrap=True, n_jobs=-1    
)
bag_clf.fit(X_train,y_train)
y_pred = bag_clf.predict(X_test)



8.3 oob 평가법
- 배깅을 사용하면 아예 선택이 안되는 샘플이 37%정도 발생
- 이를 이용하여 검증 세트로 사용하는 것
- 앙상블의 평가는 각 예측기의 oob 평가를 평균하여 얻는다.



In [7]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators = 500,
    bootstrap=True, n_jobs=-1, oob_score = True    
)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_


0.9013333333333333

8.1 랜덤 포레스트
- 훈련 샘플로 부터 무작위로 각기 다른 서브셋을 만들어 일련의 결정 트리 분류기를 훈련시킴
- 예측을 하려면 모든 개별 트리의 예측을 구하면 됩니다.


In [8]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500,max_leaf_nodes=16,n_jobs=-1)
rnd_clf.fit(X_train,y_train)

y_pred_rf = rnd_clf.predict(X_test)

500개 트리로 구성된 랜덤 포레스트로 분류 학습

트리의 노드를 분할 할 때, 특성 중에서 최선의 특성을 찾는 대신 무작위로 선택한 특성 후보 중에서 최적의 특성을 찾는 식으로 무작위성 주입